In [ ]:
#!git clone https://github.com/HumanSignal/labelImg

Cloning into 'labelImg'...
remote: Enumerating objects: 2097, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 2097 (delta 0), reused 0 (delta 0), pack-reused 2093 (from 2)
Receiving objects: 100% (2097/2097), 237.14 MiB | 4.56 MiB/s, done.
Resolving deltas: 100% (1245/1245), done.


## Augmentation will not be done due to lack of time.

## Dataset Annotation completed with LabelImg.

## dataset.yaml set up.

In [ ]:
from ultralytics import YOLO

# Load YOLO model
model = YOLO("yolov8s.pt")  # Use "yolov8n.pt" for a smaller model

# Train the model using M1 GPU
results = model.train(
    data="./dataset/dataset.yaml",
    epochs=200,
    batch=4,
    imgsz=640,
    workers=4,
    device="mps"  # Use Apple Metal GPU (M1/M2 Macs)
)

# Save the trained model
model.save("MeleeModel_v1.pt")

In [ ]:
# Import necessary libraries
from ultralytics import YOLO
import os
import cv2
import numpy as np
from glob import glob
from pathlib import Path
from collections import Counter

# Define paths
model_path = 'runs/detect/train5/weights/best.pt'  # Path to your custom model
source_folder = '/Users/rajin/Developer/ITNSHIP/RETAIL-ANALYSIS/Main/dataset/test/images'  # Source folder containing images
save_folder = '/Users/rajin/Developer/ITNSHIP/RETAIL-ANALYSIS/Main/results'  # Destination folder for results

# Parameters for customization
label_chars = 15  # Number of characters to show in the label (set to -1 for full label)
font_size = 0.4   # Font size for labels
line_thickness = 1  # Thickness of bounding boxes
box_color = (0, 255, 0)  # Green color for boxes (BGR format)

# Create the destination folder if it doesn't exist
os.makedirs(save_folder, exist_ok=True)

# Load your custom model
model = YOLO(model_path)
print(f"Model loaded from {model_path}")

# Get all image paths
image_paths = []
for ext in ['.jpg', '.jpeg', '.png', '.bmp']:
    image_paths.extend(glob(os.path.join(source_folder, f"*{ext}")))

print(f"Found {len(image_paths)} images in {source_folder}")

# Process each image
for img_path in image_paths:
    # Get the filename
    filename = os.path.basename(img_path)
    
    # Run inference
    results = model(img_path)
    
    # Load the original image
    img = cv2.imread(img_path)
    
    # Extract boxes, confidence and class info
    boxes = results[0].boxes.xyxy.cpu().numpy()
    confidences = results[0].boxes.conf.cpu().numpy()
    class_ids = results[0].boxes.cls.cpu().numpy().astype(int)
    
    # Get class names
    class_names = results[0].names
    
    # Count occurrences of each class
    class_counter = Counter([class_names[cls_id] for cls_id in class_ids])
    
    # Draw custom boxes
    for box, conf, cls_id in zip(boxes, confidences, class_ids):
        x1, y1, x2, y2 = box.astype(int)
        
        # Draw a thin rectangle
        cv2.rectangle(img, (x1, y1), (x2, y2), box_color, line_thickness)
        
        # Get full class name
        class_name = class_names[cls_id]
        
        # Create label with controlled length
        if label_chars > 0:
            display_name = class_name[:label_chars]
            if len(class_name) > label_chars:
                display_name += "..."
        else:
            display_name = class_name
            
        label = f"{display_name}: {conf:.2f}"
        
        # Get text size
        text_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, font_size, 1)[0]
        
        # Draw a small background rectangle for the label
        cv2.rectangle(img, (x1, y1-text_size[1]-5), (x1+text_size[0], y1), box_color, -1)
        
        # Put text with small font size
        cv2.putText(img, label, (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, font_size, (0, 0, 0), 1)
    
    # Add class counts to the top of the image
    y_pos = 30
    cv2.putText(img, "Detections:", (10, y_pos), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    y_pos += 30
    
    for class_name, count in class_counter.items():
        text = f"{class_name}: {count}"
        cv2.putText(img, text, (10, y_pos), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
        y_pos += 25
    
    # Save the result image
    save_path = os.path.join(save_folder, filename)
    cv2.imwrite(save_path, img)
    
    print(f"Processed: {filename} - Detected {sum(class_counter.values())} objects of {len(class_counter)} classes")

print(f"All images processed. Results saved to {save_folder}")

Model loaded from runs/detect/train5/weights/best.pt
Found 150 images in /Users/rajin/Developer/ITNSHIP/RETAIL-ANALYSIS/Main/data/images

image 1/1 /Users/rajin/Developer/ITNSHIP/RETAIL-ANALYSIS/Main/data/images/horlicks4_aug_4.jpg: 640x512 41 HorlicksCTSL18mls, 142.6ms
Speed: 3.7ms preprocess, 142.6ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 512)
Processed: horlicks4_aug_4.jpg - Detected 41 objects of 1 classes

image 1/1 /Users/rajin/Developer/ITNSHIP/RETAIL-ANALYSIS/Main/data/images/doveconditioner7_aug_1.jpg: 480x640 1 HorlicksCTSL18ml, 47 DoveConditionerIRPDOLCE7mls, 142.8ms
Speed: 4.1ms preprocess, 142.8ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
Processed: doveconditioner7_aug_1.jpg - Detected 48 objects of 2 classes

image 1/1 /Users/rajin/Developer/ITNSHIP/RETAIL-ANALYSIS/Main/data/images/horlicks2_aug_3.jpg: 480x640 12 HorlicksCTSL18mls, 1 DoveConditionerIRPDOLCE7ml, 98.1ms
Speed: 2.9ms preprocess, 98.1ms inference, 0.4ms postprocess

In [2]:
# Import necessary libraries
from ultralytics import YOLO
import os
import cv2
import numpy as np
from glob import glob
from pathlib import Path
from collections import Counter

# Define paths
model_path = 'models/UBLModel_v2.pt'  # Path to your custom model
source_folder = '/Users/rajin/Developer/ITNSHIP/RETAIL-ANALYSIS/Main/data/images'  # Source folder containing images
save_folder = '/Users/rajin/Developer/ITNSHIP/RETAIL-ANALYSIS/Main/selected_results_v2'  # Destination folder for results

# Parameters for customization
label_chars = 15  # Number of characters to show in the label (set to -1 for full label)
font_size = 0.4   # Font size for labels
line_thickness = 1  # Thickness of bounding boxes
box_color = (0, 255, 0)  # Green color for boxes (BGR format)

# Create the destination folder if it doesn't exist
os.makedirs(save_folder, exist_ok=True)

# Load your custom model
model = YOLO(model_path)
print(f"Model loaded from {model_path}")

# Get all image paths
image_paths = []
for ext in ['.jpg', '.jpeg', '.png', '.bmp']:
    image_paths.extend(glob(os.path.join(source_folder, f"*{ext}")))

print(f"Found {len(image_paths)} images in {source_folder}")

# Process each image
for img_path in image_paths:
    # Get the filename
    filename = os.path.basename(img_path)
    
    # Run inference
    results = model(img_path)
    
    # Load the original image
    img = cv2.imread(img_path)
    
    # Extract boxes, confidence and class info
    boxes = results[0].boxes.xyxy.cpu().numpy()
    confidences = results[0].boxes.conf.cpu().numpy()
    class_ids = results[0].boxes.cls.cpu().numpy().astype(int)
    
    # Get class names
    class_names = results[0].names
    
    # Count occurrences of each class
    class_counter = Counter([class_names[cls_id] for cls_id in class_ids])
    
    # Draw custom boxes
    for box, conf, cls_id in zip(boxes, confidences, class_ids):
        x1, y1, x2, y2 = box.astype(int)
        
        # Draw a thin rectangle
        cv2.rectangle(img, (x1, y1), (x2, y2), box_color, line_thickness)
        
        # Get full class name
        class_name = class_names[cls_id]
        
        # Create label with controlled length
        if label_chars > 0:
            display_name = class_name[:label_chars]
            if len(class_name) > label_chars:
                display_name += "..."
        else:
            display_name = class_name
            
        label = f"{display_name}: {conf:.2f}"
        
        # Get text size
        text_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, font_size, 1)[0]
        
        # Draw a small background rectangle for the label
        cv2.rectangle(img, (x1, y1-text_size[1]-5), (x1+text_size[0], y1), box_color, -1)
        
        # Put text with small font size
        cv2.putText(img, label, (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, font_size, (0, 0, 0), 1)
    
    # Add class counts to the top of the image
    y_pos = 30
    cv2.putText(img, "Detections:", (10, y_pos), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    y_pos += 30
    
    for class_name, count in class_counter.items():
        text = f"{class_name}: {count}"
        cv2.putText(img, text, (10, y_pos), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
        y_pos += 25
    
    # Save the result image
    save_path = os.path.join(save_folder, filename)
    cv2.imwrite(save_path, img)
    
    print(f"Processed: {filename} - Detected {sum(class_counter.values())} objects of {len(class_counter)} classes")

print(f"All images processed. Results saved to {save_folder}")

Model loaded from models/UBLModel_v2.pt
Found 150 images in /Users/rajin/Developer/ITNSHIP/RETAIL-ANALYSIS/Main/data/images

image 1/1 /Users/rajin/Developer/ITNSHIP/RETAIL-ANALYSIS/Main/data/images/horlicks4_aug_4.jpg: 640x512 40 HorlicksCTSL18mls, 120.4ms
Speed: 1.7ms preprocess, 120.4ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 512)
Processed: horlicks4_aug_4.jpg - Detected 40 objects of 1 classes

image 1/1 /Users/rajin/Developer/ITNSHIP/RETAIL-ANALYSIS/Main/data/images/doveconditioner7_aug_1.jpg: 480x640 48 DoveConditionerIRPDOLCE7mls, 121.5ms
Speed: 3.2ms preprocess, 121.5ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)
Processed: doveconditioner7_aug_1.jpg - Detected 48 objects of 1 classes

image 1/1 /Users/rajin/Developer/ITNSHIP/RETAIL-ANALYSIS/Main/data/images/horlicks2_aug_3.jpg: 480x640 12 HorlicksCTSL18mls, 121.5ms
Speed: 2.1ms preprocess, 121.5ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)
Processed: horlicks2_aug